In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

In [3]:
df = pd.read_parquet('~/Data/lendingclub.parquet')

In [6]:
df.dtypes

grade                 object
installment           object
loan_amnt            float64
funded_amnt           object
home_ownership        object
mort_acc              object
pub_rec               object
settlement_status     object
dtype: object

In [11]:
df['loan_amnt'] = df['loan_amnt'].astype('float')
df['funded_amnt'] = df['funded_amnt'].astype('float')
df['installment'] = df['installment'].astype('float')
df['pub_rec'] = df['pub_rec'].astype('float')

In [14]:
df = df.drop('mort_acc', axis=1)

In [19]:
for v in ['loan_amnt', 'funded_amnt', 'installment', 'pub_rec']:
    df[v] = (df[v]-df[v].mean()) / df[v].std()



In [24]:
dumdf = pd.get_dummies(df).fillna(0)

In [25]:
df.groupby('settlement_status')['settlement_status'].count()

settlement_status
ACTIVE    14811
BROKEN     4728
Name: settlement_status, dtype: int64

In [ ]:
dumdf = dumdf.sample(frac=1)

In [26]:
dumdf.dtypes

installment                 float64
loan_amnt                   float64
funded_amnt                 float64
pub_rec                     float64
grade_A                       uint8
grade_B                       uint8
grade_C                       uint8
grade_D                       uint8
grade_E                       uint8
grade_F                       uint8
grade_G                       uint8
home_ownership_ANY            uint8
home_ownership_MORTGAGE       uint8
home_ownership_OWN            uint8
home_ownership_RENT           uint8
settlement_status_ACTIVE      uint8
settlement_status_BROKEN      uint8
dtype: object

In [27]:
X  = dumdf.iloc[:,:-2]
y = dumdf.iloc[:,-2]

X_train, X_test, y_train, y_test = X.iloc[:15000], X.iloc[15000:], y.iloc[:15000], y.iloc[15000:]

In [28]:
knn = KNeighborsClassifier(n_neighbors=5 )
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [29]:
y_pred = knn.predict(X_test)

In [30]:
from sklearn.metrics import confusion_matrix

In [31]:
cm = confusion_matrix(y_test, y_pred)

In [39]:
(cm[0][0]+cm[1][1])/cm.sum()

0.6935448336638026

In [35]:
cm

array([[ 114, 1088],
       [ 303, 3034]])

In [41]:
1 - 4728/len(dumdf)

0.7580224167050514

Vi får med andre ord et påfallende dårlig resultat.

## SVM-baserte modeller

In [42]:
from sklearn import svm

In [43]:
lc_svm = svm.SVC()

In [44]:
lc_svm.fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [45]:
y_pred = lc_svm.predict(X_test)

In [46]:
cm = confusion_matrix(y_test, y_pred)

In [47]:
(cm[0][0]+cm[1][1])/cm.sum()

0.7351839612249395

## Regresjon

In [48]:
from sklearn.linear_model import LogisticRegression

In [49]:
lc_logreg = LogisticRegression().fit(X_train, y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [50]:
y_pred = lc_logreg.predict(X_test)

In [51]:
cm = confusion_matrix(y_test, y_pred)

In [52]:
(cm[0][0]+cm[1][1])/cm.sum()

0.7351839612249395

In [54]:
y_test.mean()

0.7351839612249395